### Imports

In [35]:
import librosa
import pandas as pd
import numpy as np
import noisereduce as nr
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import re
import pyarabic.araby as araby
from pyarabic.normalize import normalize_hamza as normalize_text
import string


### Reading dataset

In [36]:
df=pd.read_csv('Dataset/index.csv')
df.head(3)

,audio_file,text,gender
0,D6x81u_1.mp3,خليني ابدا الفيديو ده وانا بقول ان من,male
1,6f5WmH_2.mp3,ااا ملاحظتي الشخصيه استنتجت ان الستات ما,male
2,abSOwC_3.mp3,بقاش عندها دم,male


### Preprocessing

In [37]:
def preprocess_arabic_text(text):
    # Remove diacritics (tashkeel)
    text = araby.strip_tashkeel(text)
    
    # Normalize Arabic text (standardize different forms of letters)
    text = normalize_text(text)
    
    # Remove punctuation and special characters
    arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!"…"–ـ'''
    all_punctuations = string.punctuation + arabic_punctuations
    translator = str.maketrans('', '', all_punctuations)
    text = text.translate(translator)
    
    # Remove extra spaces
    text = re.sub(' +', ' ', text)
    
    return text.strip()

In [38]:
audio_files = set(os.listdir( 'Dataset/data/')) 
csv_audio_ids = set(df['audio_file'].values)
audio_without_transcription = audio_files - csv_audio_ids
print("Audio files without transcriptions:", audio_without_transcription)
transcription_without_audio = csv_audio_ids - audio_files
print("Transcriptions without audio files:", transcription_without_audio)
df.drop(df[df['audio_file']=='لشخصك ولأفكارك الـProgressive،"'].index,inplace = True)
df.drop(df[df['text'] == '[موسيقى]'].index,inplace = True)

Audio files without transcriptions: set()
Transcriptions without audio files: {'لشخصك ولأفكارك الـProgressive،"'}


In [ ]:
df.set_index('audio_file', inplace=True)
df['mfccs'] = None
df['cleaned_text'] = None
df['normalized_text'] = None
error_files = [] 

for audio in tqdm(df.index, desc="Processing Audio Files"):
    audio_path = 'Dataset/data/' + audio
    try:
        signal, rate = librosa.load(audio_path, sr=16000)
        df.at[audio, 'length'] = len(signal) / rate
        cleaned_audio = nr.reduce_noise(signal, rate)
        resampled_audio = librosa.resample(cleaned_audio, orig_sr=rate, target_sr=16000)
        normalized_audio = librosa.util.normalize(resampled_audio)
        mfccs = librosa.feature.mfcc(y=normalized_audio, sr=16000, n_mfcc=13)
        df.at[audio, 'mfccs'] = mfccs.tolist()

        raw_text = df.at[audio, 'text']

        cleaned_text = preprocess_arabic_text(raw_text)
        df.at[audio, 'cleaned_text'] = cleaned_text
        error_files.append((audio, "Missing or invalid text"))      
    except Exception as e:
        error_message = str(e)
        tqdm.write(f"Error processing {audio}: {error_message}")
        os.remove(audio_path)
        df.drop(audio,axis=0,inplace=True)
        error_files.append((audio, error_message))

if error_files:
    print("\nFiles with errors:")
    for file, error in error_files:
        print(f"- {file}: {error}")
    print(f"Total files with errors: {len(error_files)}/{len(df.index)}")
else:
    print("\nAll files processed successfully!")


Processing Audio Files:   8%|▊         | 32612/405874 [1:00:16<2:11:16, 47.39it/s] /Users/maryamsaad/Library/Python/3.9/lib/python/site-packages/noisereduce/spectralgate/nonstationary.py:70: RuntimeWarning: invalid value encountered in divide
  sig_mult_above_thresh = (abs_sig_stft - sig_stft_smooth) / sig_stft_smooth
Processing Audio Files:   8%|▊         | 32623/405874 [1:00:16<2:09:41, 47.97it/s]

Error processing 8VvErN_634.mp3: Audio buffer is not finite everywhere


Processing Audio Files:  11%|█         | 44767/405874 [1:05:03<2:31:28, 39.73it/s]

Error processing IF1Lgl_1794.mp3: Audio buffer is not finite everywhere


Processing Audio Files:  23%|██▎       | 94569/405874 [3:14:20<1:27:55, 59.00it/s]   

...

Error processing pNuehK_2176.mp3: Audio buffer is not finite everywhere


Processing Audio Files:  78%|███████▊  | 317545/405874 [4:33:44<17:28, 84.25it/s]   /Users/maryamsaad/Library/Python/3.9/lib/python/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=640
  warnings.warn(
Processing Audio Files:  83%|████████▎ | 336711/405874 [4:43:55<24:59, 46.12it/s]    

Error processing ypZjew_595.mp3: Audio buffer is not finite everywhere


Processing Audio Files: 100%|██████████| 405874/405874 [5:26:40<00:00, 20.71it/s]    


All files processed successfully!


In [40]:
df.to_csv("Preprocessed_df.csv")

In [43]:
df['length'].describe()
df=df[(df['length']<10) & (df['length']>2)]



In [45]:
df.tail()

,text,gender,mfccs,cleaned_text,normalized_text,length
audio_file,,,,,,
Onv0cx_252.mp3,القناه ولو استفدت من الحلقه دي شاركها مع,female,"[[-332.20611572265625, -315.228271484375, -306...",القناه ولو استفدت من الحلقه دي شاركها مع,None,6.160000
eDDt24_253.mp3,اهلك واصحابك وخليهم يعرفوا اكتر كل,female,"[[-353.6945495605469, -368.27996826171875, -40...",اهلك واصحابك وخليهم يعرفوا اكتر كل,None,5.720000
GkG8mp_254.mp3,امنياتي بالشفاء والقوه لكل محاربين,female,"[[-287.0047912597656, -219.3107147216797, -198...",امنياتي بالشفاء والقوه لكل محاربين,None,4.130000
0VO9mA_255.mp3,الامراض في كل,female,"[[-242.59896850585938, -290.6256408691406, -36...",الامراض في كل,None,5.279000
OrpHs8_257.mp3,مكان,female,"[[-103.92257690429688, -56.92582321166992, -10...",مكان,None,5.281062


#### Text Preprocessing

In [ ]:
import torch
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS device")
else:
    device = torch.device("cpu")
    print("MPS device not found, using CPU")

Using MPS device


In [63]:
from sentence_transformers import SentenceTransformer
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  


if torch.backends.mps.is_available():
    model = model.to(torch.device("mps"))


def get_sentence_embedding(text):
    embedding = model.encode(text)
    return embedding


def semantic_similarity(text1, text2):
    emb1 = get_sentence_embedding(text1)
    emb2 = get_sentence_embedding(text2)
    return cosine_similarity([emb1], [emb2])[0][0]


RuntimeError: Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
empty_like method already has a different docstring